In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
#LLM we will be using 
llm_model= "gpt-3.5-turbo-0301"

Now we will write a function that takes the input of the user and model, pass the input to model (which is our LLM) and return the response of the LLM.

In [5]:
def get_completion(prompt, model=llm_model):
    #messagge---> User role means that this is the message of the user
    messages = [{"role": "user", "content": prompt}]
    #Passing message to model and getting response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        #no randomness in the output
        temperature=0, 
    )
    #returns the response
    return response.choices[0].message["content"]


In [7]:
#lets test
get_completion("What is 1200/60")

'20'

Lets say we have the email of the customer in someother language and we want to convert them to American English. 

In [9]:
#So customer email in Pirate English
customer_email= """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [11]:
#we want it to be in American English so
required_style= """American English \
in a calm and respectful tone
"""

Now we have to make a prompt by combining the Customer Email which is input and stle so that we can pass that prompt to the model.

In [13]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {required_style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [14]:
#passing it to above function and getting response
response= get_completion(prompt)

In [15]:
print(response)

I am quite upset that my blender lid came off and caused my smoothie to splatter all over my kitchen walls. Additionally, the warranty does not cover the cost of cleaning up the mess. Would you be able to assist me at this time, my friend?


# Langchain

### What is it and Why do we need it?

LangChain is basically the Framework used for building applications with LLMs. 
Consider here in this example if you want now to respond user with American Pirate, for that you want to convert your written response which is written in American English to Pirate English.

For that You will again write Prompt from very start to Convert this and then you will send. So here Langchain comes in the scene. 

#### Prompts:
With Langchain Prompts basically you can make Prompt Template that you can reuse again and again. It saves alot of time when you have application wher prompts are similar.

So lets create Prompt Template with which we can convert these emails to the language and style we want with just giving the style and input email.

We will have a genral prompt that we will be able to use for different inputs.

In [16]:
#importing ChatOpenAI with which we will make our LLM model first (by making its object)
from langchain.chat_models import ChatOpenAI

In [18]:
#To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-0301', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

**Prompt Template for Translation of Emails**

In [19]:
#this is the template string that we will pass to form_template function of LangChain|
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [21]:
from langchain.prompts import ChatPromptTemplate

Now using form_Template to make a Prompt Template from the string we have specified above.

In [23]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [24]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

As you can see from input_variables that it requires two variable style and text.

In [25]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

Now lets specify Customer Conversion Style which will be American English.

In [26]:
#as customers email will be in other language like English Pirate and we will convert it American English
customer_style = """American English \
in a calm and respectful tone
"""

In [27]:
#now input text of email that the customer have sent
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

Now we will form the Prompt or the message by using the template of prompt we have made with form_template()

In [28]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [29]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [30]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


In [31]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [32]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?


#### Lets use same template to Convert Email from Servic Employee to Customer, with this you will get an Idea of what type of Abstraction LangChain Prompts Provide.

In [33]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [34]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

See we are just changing the Input and Style rest, Prompt remains same which was prompt_template.

In [35]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [36]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, me hearty customer! I be sorry to inform ye that the warranty be not coverin' the expenses o' cleaning yer galley, as it be yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Farewell and may the winds be in yer favor!


It is benficial when in Complex Application where you have Long Prompts similar to each other so you have to write again and again. 

So with this you can just reuse them.